# Using zip

The [zip docs](https://docs.python.org/3/library/functions.html#zip) say that `zip(*iterables, strict=False)`:

> Iterate over several iterables in parallel, producing tuples with an item from each one.

## Showing the Lists

I use fastcore's `L` to show the lists.

In [42]:
from fastcore.all import L

Otherwise the cell output is a `zip` object:

In [43]:
zip(range(2))

## Create 2 Iterables

In [44]:
l1 = L(range(5))
l1

(#5) [0,1,2,3,4]

In [45]:
l2 = L(range(6,11))
l2

(#5) [6,7,8,9,10]

## Zip the 2 Iterables Together

The most common scenario is zipping 2 same-length iterables together:

In [46]:
L(zip(l1,l2))

(#5) [(0, 6),(1, 7),(2, 8),(3, 9),(4, 10)]

## Zip a Long and Short Iterable Together

It stops when the shortest iterable is exhausted:

In [47]:
L(zip(l1,l2[3:]))

(#2) [(0, 9),(1, 10)]

## Zip 1 Iterable

With 1 iterable, you get an iterator of tuples:

In [48]:
L(zip(l1))

(#5) [(0,),(1,),(2,),(3,),(4,)]

## Zip Nothing

With no args, you get an empty iterator:

In [49]:
L(zip())

(#0) []

## Zip Dict Items

`items` gives you an iterator of tuples, each tuple being a k,v pair:

In [50]:
d = {1: "one", 2: "two", 3: "three"}
L(d.items())

(#3) [(1, 'one'),(2, 'two'),(3, 'three')]

Destructuring and zipping that transposes the tuples:

In [51]:
L(zip(*d.items()))

(#2) [(1, 2, 3),('one', 'two', 'three')]

## Application: Zip a Dict Tree

This tree is from [Jeremy Howard's bfs tweet](https://x.com/jeremyphoward/status/1873810934608912550):

In [52]:
tree = {1: {2: {4: {}}, 3: {5: {}, 6: {}}}}
L(tree)

(#1) [1]

It represents this:

```
1
├── 2
│   └── 4
└── 3
    ├── 5
    └── 6
```

Breadth-first search (BFS) would traverse the tree from shallow to deep, printing each number as: `1 2 3 4 5 6`

Jeremy implemented BFS as:

In [53]:
def bfs(q):
    while q:
        k,v = zip(*q)
        print(*k)
        q = [x for o in v for x in o.items()]

To use it, he calls it with the tree's items:

In [54]:
bfs(tree.items())

1
2 3
4 5 6


The tree's items are

In [55]:
L(tree.items())

(#1) [(1, {2: {4: {}}, 3: {5: {}, 6: {}}})]

In [22]:
L(zip(*tree.items()))

(#2) [(1,),({2: {4: {}}, 3: {5: {}, 6: {}}},)]

Current node:

In [23]:
L(zip(*tree.items()))[0]

(1,)

Child nodes:

In [24]:
L(zip(*tree.items()))[1]

({2: {4: {}}, 3: {5: {}, 6: {}}},)

### Why zip in bfs()?

The `zip` in `k,v = zip(*q)` serves to separate node values from their children at each level. Let's see how:

1. First iteration:
   ```python
   q = [(1, {2: {4: {}}, 3: {5: {}, 6: {}}})]
   k,v = zip(*q)  # k=(1,), v=({2: {4: {}}, 3: {5: {}, 6: {}}},)
   ```

2. Second iteration:
   ```python
   q = [(2, {4: {}}), (3, {5: {}, 6: {}})]
   k,v = zip(*q)  # k=(2,3), v=({4: {}}, {5: {}, 6: {}})
   ```

3. Third iteration:
   ```python
   q = [(4, {}), (5, {}), (6, {})]
   k,v = zip(*q)  # k=(4,5,6), v=({},{},{})
   ```

The `zip` cleanly separates values (`k`) to print from their children (`v`) needed for the next level of traversal.